# Importando do GCP

## Tratando Dados

In [ ]:
!pip install PyMySQL

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 8.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [ ]:
def create_connection():
    connection = pymysql.connect(
        host='host',      # Substitua pelo seu host
        user='user',  # Substitua pelo seu usuário
        password='password',  # Substitua pela sua senha
        database='recomenda_database'  # Substitua pelo nome do seu banco de dados
    )
    return connection

In [ ]:
def get_table_data():
    connection = create_connection()
    query = "SELECT * FROM imdb_database order by IMDB_Rating DESC; "
    df = pd.read_sql(query, connection)
    connection.close()
    return df

df = get_table_data()
#df.head()

<ipython-input-5-0014cf6f8639>:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [ ]:
# Recebe csv do IMDB e transforma os dados, pegando as informações relevantes para nosso trabalho e criando novos arquivos, cada um para um tipo de nó que será futuramente criado no NEO4J
import pandas as pd
from unidecode import unidecode

# Listas para armazenar dados extraídos
filmes = []
diretores = []
atores = []
generos = []

# Remove as colunas que não serão utilizadas
df = df.drop(columns=['Gross', 'Runtime', 'No_of_Votes', 'Overview', 'Meta_score', 'Poster_Link'])

# Renomeia a coluna 'Series_Title' para 'Title'
df = df.rename(columns={'Series_Title': 'Title'})
df.head()

,Title,Released_Year,Certificate,Genre,IMDB_Rating,Director,Star1,Star2,Star3,Star4
0,The Shawshank Redemption,1994,A,Drama,9.3,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler
1,The Godfather,1972,A,"Crime, Drama",9.2,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton
2,The Dark Knight,2008,UA,"Action, Crime, Drama",9.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine
3,The Godfather: Part Ii,1974,A,"Crime, Drama",9.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton
4,12 Angry Men,1957,U,"Crime, Drama",9.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler


In [ ]:
# Função para classificar os filmes de acordo com a classificação indicativa
def classifica(classificacao):
    classificacoes = {
        'A': '18', 'UA': '12', 'U': 'L', 'PG-13': '13', 'R': '16',
        'PG': '10', 'G': 'L', 'Passed': 'L', 'TV-14': '16', '16': '16',
        'TV-MA': '16', 'Unrated': 'Unrated', 'GP': 'L', 'Approved': 'L',
        'TV_PG': '10', 'U/A': '10'
    }
    return classificacoes.get(classificacao, 'Unrated')

# Função para verificar e adicionar atores únicos à lista
def verifica_ator(atores, ator1, ator2, ator3, ator4):
    for ator in [ator1, ator2, ator3, ator4]:
        ator = unidecode(ator).replace("'", '')
        if ator not in atores:
            atores.append(ator)

# Função para verificar e adicionar diretores únicos à lista
def verifica_diretor(diretores, dir):
    dir = unidecode(dir).replace("'", '')
    if dir not in diretores:
        diretores.append(dir)

# Função para verificar e adicionar filmes únicos à lista
def verifica_filme(filmes, filme, ano, rating, classificacao):
    filme = unidecode(filme).replace("'", "").replace('"', '')
    cert = classifica(classificacao)
    if filme[0].isnumeric():
        titulo = f'The{filme}_{ano}_{rating}_{cert}'
    else:
        titulo = f'{filme}_{ano}_{rating}_{cert}'
    if titulo not in filmes:
        filmes.append(titulo)

# Função para verificar e adicionar gêneros únicos à lista
def verifica_genero(generos, genres):
    for genre in genres.split(','):
        genre = unidecode(genre.strip())
        if genre not in generos:
            generos.append(genre)

# Processa cada linha do DataFrame
for indice, linha in df.iterrows():
    verifica_filme(filmes, linha['Title'], linha['Released_Year'], linha['IMDB_Rating'], linha['Certificate'])
    verifica_diretor(diretores, linha['Director'])
    verifica_ator(atores, linha['Star1'], linha['Star2'], linha['Star3'], linha['Star4'])
    verifica_genero(generos, linha['Genre'])
    #if indice >= 150:
     #   break

# Divide a lista de filmes em colunas separadas
filmes = [filme.split("_") for filme in filmes]
filmes

[['The Shawshank Redemption', '1994', '9.3', '18'],
 ['The Godfather', '1972', '9.2', '18'],
 ['The Dark Knight', '2008', '9.0', '12'],
 ['The Godfather: Part Ii', '1974', '9.0', '18'],
 ['The12 Angry Men', '1957', '9.0', 'L'],
 ['Pulp Fiction', '1994', '8.9', '18'],
 ['Schindlers List', '1993', '8.9', '18'],
 ['The Lord Of The Rings: The Return Of The King', '2003', '8.9', 'L'],
 ['The Lord Of The Rings: The Fellowship Of The Ring', '2001', '8.8', 'L'],
 ['Fight Club', '1999', '8.8', '18'],
 ['Forrest Gump', '1994', '8.8', '12'],
 ['Il Buono, Il Brutto, Il Cattivo', '1966', '8.8', '18'],
 ['Inception', '2010', '8.8', '12'],
 ['Goodfellas', '1990', '8.7', '18'],
 ['One Flew Over The Cuckoos Nest', '1975', '8.7', '18'],
 ['Star Wars: Episode V - The Empire Strikes Back', '1980', '8.7', '12'],
 ['The Lord Of The Rings: The Two Towers', '2002', '8.7', '12'],
 ['The Matrix', '1999', '8.7', '18'],
 ['Saving Private Ryan', '1998', '8.6', '16'],
 ['Se7En', '1995', '8.6', '18'],
 ['Sen To Chih

In [ ]:
# Cria DataFrames a partir das listas
df_filme = pd.DataFrame(filmes, columns=['Title', 'Released_Year', 'Rating', 'Certificate'])
df_diretor = pd.DataFrame(diretores, columns=['Nome'])
df_ator = pd.DataFrame(atores, columns=['Nome'])
df_genero = pd.DataFrame(generos, columns=['Genero'])

df_filme.head()
# Salva os DataFrames resultantes em arquivos CSV
df_filme.to_csv('filmes.csv', index=False, encoding='utf-8')
df_diretor.to_csv('diretores.csv', index=False, encoding='utf-8')
df_ator.to_csv('atores.csv', index=False, encoding='utf-8')
df_genero.to_csv('generos.csv', index=False, encoding='utf-8')

# Imprime o DataFrame resultante após as operações de limpeza e renomeação
# print(df_filme)

In [ ]:
# Cria os comandos Cypher para criação dos nós e relacionamentos que serão inseridos
import pandas as pd
from unidecode import unidecode

# Função para criar e escrever dados em um arquivo texto
def cria_arquivo(nome, arq):
    nome_do_arquivo = nome + '.txt'
    with open(nome_do_arquivo, 'w') as arquivo:
        arquivo.write(arq)
    print(f"Dados foram escritos no arquivo {nome_do_arquivo} com sucesso.")

# Leitura dos arquivos CSV
filmes = pd.read_csv('filmes.csv')
generos = pd.read_csv('generos.csv')
diretores = pd.read_csv('diretores.csv')
atores = pd.read_csv('atores.csv')
filmes['Title'] = filmes['Title'].apply(str.title)

# Criação dos comandos Cypher para filmes
a = ''
for indice, linha in filmes.iterrows():
    titulo = linha['Title']
    tit = titulo.replace(' ', '').replace(',', '').replace('-', '').replace('.', '').replace('*', '').replace('!', '').replace('?', '').replace('/', '').replace('&', '').replace('(', '').replace(')', '').replace(':', '').replace("'", "")

    if tit[0].isnumeric():
        tit = 'The' + tit
    tit = tit + str(linha['Released_Year'])

    a += (f"MERGE ({tit}:Movie {{title:'{titulo}'}}) "
          f"ON CREATE SET {tit}.year='{linha['Released_Year']}', "
          f"{tit}.certificate='{linha['Certificate']}', "
          f"{tit}.rating='{linha['Rating']}'\n")

cria_arquivo('neo4j_filme', a)

# Criação dos comandos Cypher para gêneros
a = ''
for indice, linha in generos.iterrows():
    tit = linha['Genero'].replace('-', '')
    a += (f"MERGE ({tit}:Genre {{name:'{tit}'}})\n")

cria_arquivo('neo4j_genero', a)

# Criação dos comandos Cypher para atores
a = ''
for indice, linha in atores.iterrows():
    tit = linha['Nome']
    titulo = tit
    tit = tit.replace(' ', '').replace("'", "").replace('.', '').replace('-', '')
    a += (f"MERGE ({tit}:Actor {{name:'{titulo}'}})\n")

cria_arquivo('neo4j_ator', a)

# Criação dos comandos Cypher para diretores
a = ''
for indice, linha in diretores.iterrows():
    tit = linha['Nome']
    titulo = tit
    tit = tit.replace(' ', '').replace("'", "").replace('.', '').replace('-', '')
    a += (f"MERGE ({tit}:Director {{name:'{titulo}'}})\n")

cria_arquivo('neo4j_diretor', a)

# Criação dos comandos Cypher para relacionamentos
cont = 0
cont2 = 0
a = ''

for indice, linha in df.iterrows():
    ator1 = unidecode(linha['Star1'].replace("'", ""))
    ator2 = unidecode(linha['Star2'].replace("'", ""))
    ator3 = unidecode(linha['Star3'].replace("'", ""))
    ator4 = unidecode(linha['Star4'].replace("'", ""))
    diretor = unidecode(linha['Director'].replace("'", ""))
    tit = unidecode(linha['Title'].title()).replace("'", "")

    a += (f"MATCH (movie:Movie {{title: '{tit}'}})\n"
          f"MATCH (actor1:Actor {{name: '{ator1}'}})\n"
          f"MATCH (actor2:Actor {{name: '{ator2}'}})\n"
          f"MATCH (actor3:Actor {{name: '{ator3}'}})\n"
          f"MATCH (actor4:Actor {{name: '{ator4}'}})\n"
          f"MATCH (dir:Director {{name: '{diretor}'}})\n")

    generos = linha['Genre'].split(',')
    lista_genero = []
    for genero in generos:
        genero = unidecode(genero.replace('-', '').replace(' ', ''))
        a += f"MATCH (genre{genero}:Genre {{name: '{genero}'}})\n"
        lista_genero.append(genero)

    for i in lista_genero:
        a += f'MERGE (movie)-[:TEM_GENERO]->(genre{i})\n'

    a += (f'MERGE (actor1)-[:ATUA_EM]->(movie)\n'
          f'MERGE (actor2)-[:ATUA_EM]->(movie)\n'
          f'MERGE (actor3)-[:ATUA_EM]->(movie)\n'
          f'MERGE (actor4)-[:ATUA_EM]->(movie)\n'
          f'MERGE (dir)-[:DIRIGE]->(movie)\n')

    cont += 1
    cont2 += 1
    if cont >= 1:
        a += '\n'
        cont = 0
    #if cont2 >= 10:
    #    break

cria_arquivo('neo4j_relacionamentos', a)


Dados foram escritos no arquivo neo4j_filme.txt com sucesso.
Dados foram escritos no arquivo neo4j_genero.txt com sucesso.
Dados foram escritos no arquivo neo4j_ator.txt com sucesso.
Dados foram escritos no arquivo neo4j_diretor.txt com sucesso.
Dados foram escritos no arquivo neo4j_relacionamentos.txt com sucesso.


## Adicionando Dados Faltantes

# Neo4j

## Cria os Nós

In [ ]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.7/293.7 kB 5.7 MB/s eta 0:00:00


In [ ]:
from neo4j import GraphDatabase
import time
import math

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "URI"
USERNAME = "neo4j"
PASSWORD = "PASSWORD"
AUTH = (USERNAME, PASSWORD)

def conecta_neo4j(query):
    with open('nome_do_arquivo.txt', 'a') as arquivo:
            arquivo.write('nome_do_arquivo.txt')
    driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))
    with driver.session() as session:
        result = session.run(query)
        records = list(result)  # Consumir todos os registros
        return records
    driver.close()  # Fechar o driver

# Verificar conectividade
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()


In [ ]:
a = ["CREATE CONSTRAINT unique_movie_title_year IF NOT EXISTS FOR (m:Movie) REQUIRE (m.title, m.year) IS UNIQUE",
"CREATE CONSTRAINT director_name IF NOT EXISTS FOR (d:Director) REQUIRE d.name IS UNIQUE",
"CREATE CONSTRAINT actor_name IF NOT EXISTS FOR (a:Actor) REQUIRE a.name IS UNIQUE",
"CREATE CONSTRAINT genre_name IF NOT EXISTS FOR (g:Genre) REQUIRE g.name IS UNIQUE",
"CREATE CONSTRAINT user_name IF NOT EXISTS FOR (u:User) REQUIRE u.name IS UNIQUE"]

for x in range (len(a)):
    conecta_neo4j(a[x])

print("Nós Criados [Teoricamente]")

# cria = ["neo4j_diretor.txt","neo4j_filme.txt", "neo4j_genero.txt","neo4j_ator.txt", "neo4j_relacionamentos.txt"]
cria = ["neo4j_genero.txt", "neo4j_relacionamentos.txt"]

for x in range (len(cria)):
    print(f"Abrindo {cria[x]}: \n")
    with open(cria[x], 'r') as file:
        texto = file.readlines()

    tamanho = len(texto)
    texto_formatado = ('').join(texto)

    resto = tamanho % 200
    rang = math.ceil(tamanho/200)
    if cria[x] != "neo4j_relacionamentos.txt":
        for z in range(rang):
            a = ''
            if (z != rang - 1) or (resto == 0):
                a += ('').join(texto[(z*200):((z+1)*200)]    )
                a += '\n\n'
            else:
                a += ('').join(texto[(z*200):((z*200)+resto)])
            conecta_neo4j(a)
            print(f'Teoricamente Enviado [{z+1}/{rang}]')
            #time.sleep(3)

    else:
        z = 1
        consultas = texto_formatado.split('\n\n')


        for y in range(len(consultas)):
            #print(consultas[y])
            conecta_neo4j(consultas[y])
            #time.sleep(2)
        z += 1
    #continuar = input('CONTINUAR ? ')
#    print('')
    #time.sleep(3)

## Cria Usuarios e os Envia para o GCP

In [ ]:
def execute_cypher_query(uri, username, password, query):
    driver = GraphDatabase.driver(uri, auth=(username, password))
    with driver.session() as session:
        result = session.run(query)
        records = list(result)  # Consumir todos os registros
        return records
    driver.close()  # Fechar o driver


def enviar(texto):
    cypher_query = texto     # Comando Cypher para recomendação de filmes
    records = execute_cypher_query(URI, USERNAME, PASSWORD, cypher_query)     # Executar o comando Cypher
    return records   # Processar e imprimir o resultado

In [ ]:
from sqlalchemy import create_engine, text

host='host'
user='user'
password='password'
database='recomenda_database'

def create_connection2():
    # Substitua pela URL de conexão com seu banco de dados
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')
    return engine.connect()

def adiciona_usuario_sql(nome):
    connection = create_connection2()
    query = text(f"INSERT INTO users VALUES ('{nome}')")
    with connection.begin() as transaction:
        connection.execute(query)
    connection.close()

def adiciona_filme_sql (nome, filmes):
    connection = create_connection2()
    query = "INSERT INTO assistiu VALUES "
    values = []

    for filme in filmes:
        values.append(f"('{nome}', '{filme}')")

    query += ", ".join(values) + ";"
    query = text(query)
    #print(query)
    with connection.begin() as transaction:
        connection.execute(query)

    connection.close()

def adiciona_recomendacao_sql(nome, filmes, score):
    connection = create_connection2()
    query = "INSERT INTO recomendado VALUES "
    values = []

    cont = 0
    for filme in filmes:
        values.append(f"('{nome}', '{filme}', '{score[cont]}') ")
        cont += 1

    query += ", ".join(values) + ";"
    query = text(query)

    with connection.begin() as transaction:
        connection.execute(query)

    connection.close()

In [ ]:
def verifica_usuario(nome):
  connection = create_connection()
  query = "SELECT username FROM users;"
  users = pd.read_sql(query, connection)
  connection.close()
  users = users['username'].tolist()
  if nome in users:
    print("\033[31mEsse nome de usuário já existe. Tente Novamente.\033[m")
    return 0
  else:
    query = f"MERGE ({usuario}:User {{nome:'{usuario}'}})"
    enviar(query)
    return 1

In [ ]:
def adicionar_filmes(usuario, filmes):
    cont2 = 0
    query = f"MATCH (user:User {{nome:'{usuario}'}})"
    for filme in filmes:
        query += f"MATCH (filme{cont2}:Movie {{title:'{filme}'}})"
        cont2 += 1
    cont2 = 0
    for filme in filmes:
        query += f"MERGE (user)-[:ASSISTIU]->(filme{cont2})"
        cont2 += 1
    #print(query)
    a = enviar(query)
    #print(a)

In [ ]:
def recomendacao(usuario):
    filmes_recomendados = []
    score = []
    query = f"""
        MATCH (usuario:User {{nome: '{usuario}'}})-[:ASSISTIU]->(filme:Movie)
        WITH filme, usuario
        MATCH (filme)-[:ATUA_EM|DIRIGE|TEM_GENERO]-(relacionado)
        MATCH (relacionado)-[:ATUA_EM|DIRIGE|TEM_GENERO]-(recomendacao:Movie)
        MATCH (d:Director)-->(recomendacao)-->(g:Genre)
        WHERE NOT (usuario)-[:ASSISTIU]->(recomendacao)
        RETURN recomendacao, COUNT(*) as score, d.name as diretor, COLLECT (DISTINCT g.name) as genero
        ORDER BY score DESC
        LIMIT 10
        """
    #print(query)
    records = enviar(query)

    print('Aqui estão alguns filmes que você pode gostar:')
    cont = 0
    print(f"\t Pos | {'Filme':^30} - {'Diretor':^30} - {'Ano':^10} - {'Genero Principal':^15} - {'Nota':^15} - {'Proximidade':^15}")
    for record in records:
        filmes_recomendados.append(record['recomendacao']['title'])
        score.append(record['score'])
        if (len(record['recomendacao']['title']) > 25):
            rec = record['recomendacao']['title'][0:25] + "..."
        else:
            rec = record['recomendacao']['title']
        print(f"\t {cont+1:^2}º | {rec:^30} - {record['diretor']:^30} - {record['recomendacao']['year']:^10} - {record['genero'][0]:^15} - {record['recomendacao']['rating']:^15} - {record['score']:^15}")
        cont += 1

    adiciona_recomendacao_sql(usuario, filmes_recomendados, score)

In [ ]:
Cdf_filmografia = df_filme[['Title', 'Released_Year']]
filmografia = df_filme['Title'].tolist()

while True:
  usuario = input('Insira seu nome de usuário: ')
  if usuario == "0":
    break
  existe = verifica_usuario(usuario)
  adiciona_usuario_sql(usuario)
  if existe == 1:
    quantidade = int(input("Numero de filmes que deseja adicionar: "))
    filmes = []
    cont = 0
    print(f'Adicionando filmes para o usuário {usuario}')
    while True:
        a = (input(f"\tAdicione nome do {cont+1}º filme (Título no Idioma Original / Romanizado): ")).title()
        if a not in filmografia:
            print(f'\tFilme não Encontrado.\n\tSe a ortografia estiver correta, é provavél que esse filme não se encontra no Top 1000 IMDB\n\tPor favor, tente novamente\n')
        else:
            filmes.append(a)
            ano = df_filmografia[df_filmografia['Title'] == 'The12 Angry Men']['Released_Year'].values[0]
            cont += 1
        if cont >= quantidade:
            break
    adicionar_filmes(usuario, filmes)
    adiciona_filme_sql(usuario, filmes)
    recomendacao(usuario)

Insira seu nome de usuário: Gerson


<ipython-input-15-201dddbf3509>:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users = pd.read_sql(query, connection)


Numero de filmes que deseja adicionar: 10
Adicionando filmes para o usuário Gerson
	Adicione nome do 1º filme (Título no Idioma Original / Romanizado): the kid
	Adicione nome do 2º filme (Título no Idioma Original / Romanizado): singin in the rain
	Adicione nome do 3º filme (Título no Idioma Original / Romanizado): snatch
	Adicione nome do 4º filme (Título no Idioma Original / Romanizado): amelie
	Adicione nome do 5º filme (Título no Idioma Original / Romanizado): the great dictator
	Adicione nome do 6º filme (Título no Idioma Original / Romanizado): city lights
	Adicione nome do 7º filme (Título no Idioma Original / Romanizado): modern times
	Adicione nome do 8º filme (Título no Idioma Original / Romanizado): la vita e bella
	Adicione nome do 9º filme (Título no Idioma Original / Romanizado): the intouchables
	Adicione nome do 10º filme (Título no Idioma Original / Romanizado): back to the future
Aqui estão alguns filmes que você pode gostar:
	 Pos |             Filme              -  

ERROR:neo4j.io:Failed to write data to connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))
ERROR:neo4j.io:Failed to write data to connection IPv4Address(('e0e98921.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))


Aqui estão alguns filmes que você pode gostar:
	 Pos |             Filme              -            Diretor             -    Ano     - Genero Principal -      Nota       -   Proximidade  
	 1 º |          Toy Story 2           -         John Lasseter          -    1999    -    Adventure    -       7.9       -       75       
	 2 º |          Toy Story 3           -          Lee Unkrich           -    2010    -    Adventure    -       8.2       -       75       
	 3 º |          Toy Story 4           -          Josh Cooley           -    2019    -    Adventure    -       7.8       -       72       
	 4 º |        Kurenai No Buta         -         Hayao Miyazaki         -    1992    -    Adventure    -       7.7       -       72       
	 5 º |      Gake No Ue No Ponyo       -         Hayao Miyazaki         -    2008    -    Adventure    -       7.7       -       72       
	 6 º |              Soul              -          Pete Docter           -    2020    -    Adventure    -       8.1    